# Setup for Amazon Bedrock
* Container: `conda_python3` <BR>
* We recommend `python 3.10` or later. 
    - version check: !python -V

## 0. Materials
- Bedrock user guide
    - https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-service.html
- Step by step vidio tutorial
    - https://www.youtube.com/watch?v=ab1mbj0acDo

## 1. role setting (adding trust relationship)

### 1.1. role check

In [31]:
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [32]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20221206T163436


### 1.2. policy
- 1.1에서 확인된 롤에 아래와 같이 2개의 권한 추가

![nn](../imgs/policy.jpg)

### 1.3. policy
- 아래와 같이 인라인 정책 생성 (Add permissions - Create inline policy)
- policy name: bedrock
![nn](../imgs/inline-policy.jpg)

In [33]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "bedrock:*"
            ],
            "Resource": "*",
            "Effect": "Allow",
            "Sid": "BedrockFullAccess"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Action': ['bedrock:*'],
   'Resource': '*',
   'Effect': 'Allow',
   'Sid': 'BedrockFullAccess'}]}

### 1.4. Trust relationship
![nn](../imgs/trust-relationship.jpg)

In [34]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        },
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Principal': {'Service': 'sagemaker.amazonaws.com'},
   'Action': 'sts:AssumeRole'},
  {'Effect': 'Allow',
   'Principal': {'Service': 'bedrock.amazonaws.com'},
   'Action': 'sts:AssumeRole'}]}

## 2. Model access

### 2.1. Amazon Bedrock Console
![nn](../imgs/model-access/1.png)

### 2.2. "Get Started"
![nn](../imgs/model-access/2.png)

### 2.3. "Model access" - "Edit"
사용 모델 활성화 창
![nn](../imgs/model-access/3.png)

### 2.3. "Save Changes"
사용할 모델을 활성화 후 저장
![nn](../imgs/model-access/4.png)

## 3. Install python SDK for bedrock

In [17]:
install_needed = True

In [18]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U awscli
    !{sys.executable} -m pip install -U botocore
    !{sys.executable} -m pip install -U boto3
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U langchain #==0.0.312
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U transformers
    !{sys.executable} -m pip install -U librosa
    !{sys.executable} -m pip install -U opensearch-py
    !{sys.executable} -m pip install -U sqlalchemy==2.0.1
    !{sys.executable} -m pip install -U pypdf
    !{sys.executable} -m pip install -U spacy
    !{sys.executable} -m spacy download ko_core_news_md
    !{sys.executable} -m pip install -U ipython
    !{sys.executable} -m pip install -U ipywidgets
    !{sys.executable} -m pip install -U llmsherpa

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.9/916.9 kB 38.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.196.0-py2.py3-none-any.whl size=1223208 sha256=24e7e7f6060ae625f05d97f57c3d069c67ed4ec592ab400e42a4ff4a7cfb81e4
  Stored in directory: /home/ec2-user/.cache/pip/wheels/22/86/1b/11b1150764a78929af99b11b7789b8f3ed340d2c31d425cfe2
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.195.1
    Uninstalling sagemaker-2.195.1:
      Successfully uninstalled sagemaker-2.195.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 MB 45.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('ko_core_news_md')
  Using cached ipywidgets-8.1.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.9-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_

## 4. Check setting
모델 리스트 <BR>
![nn](../imgs/check.png)


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [3]:
import boto3
import awscli
import botocore
import langchain
from pprint import pprint
from termcolor import colored
from utils.bedrock import bedrock_info

In [4]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")
print(f"botocore version check: {botocore.__version__}")
print(f"awscli version check: {awscli.__version__}")

langchain version check: 0.0.324
boto3 version check: 1.28.72
botocore version check: 1.31.72
awscli version check: 1.29.72


**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [5]:
bedrock_region = "us-east-1"

In [6]:
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=bedrock_region,
    endpoint_url=f"https://bedrock-runtime.{bedrock_region}.amazonaws.com"
)

In [8]:
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())


== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


## 5. generation

In [9]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

### Titan

## Titan text모델은 현재 preview 서비스입니다. GA 이후 업데이트 하도록 할께요! (현재 에러발생)

In [10]:
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Titan-Text-G1"),
    client=bedrock_client,
    model_kwargs={
        "maxTokenCount":512,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
    }
)

In [11]:
prompt = "Please let us know SageMaker's advantages in 100 words"

In [12]:
#print (colored(llm(prompt), "green"))

### Claude v2

In [13]:
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=bedrock_client,
    model_kwargs={
        "max_tokens_to_sample":512,
        "stop_sequences":["\n\nHuman:"],
        "temperature":0,
        "top_p":0.9
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [14]:
prompt = "\n\nHuman: Please let us know SageMaker's advantages in 100 words \n\nAssistant:"

In [15]:
response = llm(prompt)

 Here is a 100 word summary of some of SageMaker's key advantages:

SageMaker makes it easy to build, train, and deploy machine learning models at scale. It provides pre-built algorithms and frameworks as well as fully managed infrastructure to quickly get models into production. Automated hyperparameter tuning saves time by finding the best model configurations. SageMaker Studio provides a visual IDE for the full ML workflow. And SageMaker Neo compiles models for faster inference. The service is fully managed so you don't have to worry about infrastructure management. These capabilities help data scientists be more productive and accelerate the development of machine learning applications.

* Long text to examine 100k-input

In [18]:
text_file = "../utils/sample-text/long-text-70k-tokens.txt"
with open(text_file, "r") as file:
    text = file.read()
    
prompt = "".join(["\n\nHuman: 아래 글을 요약해줘\n\n", text, "\n\nAssistant:"])
    
print (f"# characters in prompt: {len(prompt)}")

# characters in prompt: 37557


In [19]:
response = llm(prompt)

 한글은 1443년 세종대왕이 한국어를 표기하기 위해 창제한 표음 문자입니다. 

- 초기에는 자음 17자, 모음 11자로 총 28자였으나 현재는 자음 14자, 모음 10자로 총 24자를 사용합니다.

- 자음과 모음을 조합하여 한글 음절을 이룹니다. 

- 표음문자로서 한국어를 정확하고 효율적으로 표기할 수 있습니다. 

- 세종대왕은 《훈민정음》이라는 책을 통해 한글의 창제 원리를 설명하였습니다.

- 한글은 여러 시련 속에서도 한국어 표기 문자로서의 지위를 지켜왔습니다.

- 현대 한국어 표준어에서는 2,912가지 한글 음절이 사용됩니다.

- 한글은 과학성과 체계성을 바탕으로 한 우수한 문자입니다.